<a href="https://colab.research.google.com/github/Jucavilav/CienciaD-taller2/blob/main/Taller2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install pylev

In [35]:
import re
from random import randint
from seaborn import load_dataset
from datetime import datetime
from difflib import SequenceMatcher
import numpy as np
import pandas as pd
import pylev

In [ ]:
pd.set_option("display.max_columns", None)

In [30]:
pib_df = pd.read_csv("https://raw.githubusercontent.com/Jucavilav/CienciaD-taller2/main/datos/gapminder_final.csv")

In [31]:
pib_df.shape

(178, 16)

In [32]:
pib_df.sample(5)

,country,incomeperperson,alcconsumption,armedforcesrate,breastcancerper100th,co2emissions,femaleemployrate,hivrate,internetuserate,lifeexpectancy,oilperperson,polityscore,relectricperperson,suicideper100th,employrate,urbanrate
88,Brazil,4699.411262,10.08,0.721396,46.0,9.580226e+09,53.299999,0.45,40.650098,73.488,0.599652,8.0,498.165305,4.777007,64.500000,85.58
146,Libya,7885.468037,0.10,3.311674,23.4,1.414032e+09,23.200001,NaN,14.000247,74.788,NaN,-7.0,846.014923,4.667025,48.599998,77.54
26,Saudi Arabia,9425.325870,0.34,2.618296,24.7,7.861553e+09,17.700001,NaN,41.000128,73.911,4.572268,-10.0,3433.932449,6.401538,51.200001,82.42
18,"Macedonia, FYR",2221.185664,8.94,2.101467,52.1,1.688830e+08,26.799999,NaN,51.914184,74.847,NaN,9.0,NaN,7.876878,34.900002,66.90
3,Barbados,9243.587053,6.42,0.663956,62.5,3.616067e+07,60.299999,1.40,70.028599,76.835,NaN,NaN,NaN,3.108603,66.900002,39.84


In [33]:
pib_df.rename({'country' : 'Pais' , 'incomeperperson' : 'Ingresos por persona' , 'alcconsumption' : 'Consumo de alcohol' , 'armedforcesrate' : 'Tasa de fuerzas armadas' , 'breastcancerper100th' : 'Cáncer de mama100TH' , 'co2emissions' : 'Emisiones de CO2' , 'femaleemployrate' : 'Tasa de empleo femenina' , 'hivrate' : 'Tasa de VIH' , 'internetuserate' : 'Usuario de Internet' , 'lifeexpectancy' : 'Esperanza de vida' , 'oilperperson' : 'Petrolero' , 'polityscore' : 'Puntuación política' , 'relectricperperson' : 'Electricidad' , 'suicideper100th' : 'Suicidio por 100TH' , 'employrate' : 'empleo' , 'urbanrate' : 'tarifa urbana'}, axis=1)

,Pais,Ingresos por persona,Consumo de alcohol,Tasa de fuerzas armadas,Cáncer de mama100TH,Emisiones de CO2,Tasa de empleo femenina,Tasa de VIH,Usuario de Internet,Esperanza de vida,Petrolero,Puntuación política,Electricidad,Suicidio por 100TH,empleo,tarifa urbana
0,Ireland,27595.091350,14.92,0.450024,74.9,1.633779e+09,51.000000,0.2,69.770394,80.557,1.700262,10.0,2051.802338,10.365070,59.900002,61.34
1,Albania,1914.996551,7.29,1.024736,57.4,2.237473e+08,42.099998,NaN,44.989947,76.918,NaN,9.0,636.341383,7.699330,51.400002,46.72
2,Central African Rep.,239.518749,3.17,0.148659,16.5,8.338000e+06,62.900002,4.7,2.300027,48.398,NaN,-1.0,NaN,13.548420,71.300003,38.58
3,Barbados,9243.587053,6.42,0.663956,62.5,3.616067e+07,60.299999,1.4,70.028599,76.835,NaN,NaN,NaN,3.108603,66.900002,39.84
4,Italy,18982.269290,9.72,1.729346,74.4,1.900045e+10,35.799999,0.3,53.740217,81.855,1.207250,10.0,1176.385457,4.930045,46.400002,68.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,Mexico,6105.280743,8.55,0.612340,26.4,1.330450e+10,39.599998,0.3,31.050013,76.954,0.770567,8.0,431.625379,4.288574,57.900002,77.20
174,Mauritius,5182.143721,3.53,0.349544,31.6,5.947333e+07,36.299999,1.0,28.731883,73.373,NaN,10.0,NaN,8.081540,54.500000,42.48
175,Namibia,2667.246710,12.09,1.972913,24.7,2.620933e+07,37.299999,13.1,6.500823,62.465,NaN,6.0,0.000000,8.021970,42.000000,36.84
176,Comoros,336.368749,0.28,NaN,19.5,2.368667e+06,59.299999,0.1,5.098265,61.061,NaN,9.0,NaN,4.751084,68.000000,28.08


In [37]:
pib_df['country'].nunique() #Total de paises

166

In [38]:
# Registros duplicados por Pais
dupl = pib_df['country']
duplicados = dupl[dupl.duplicated()]
print("Registros duplicados:")
print(duplicados)
# Esto indica que tenemos un total 12 Paises duplicados, con lo cual se procedera con la depuracion de los paises repetidos.

Registros duplicados:
97         Luxembourg
5         Switzerland
0             Ireland
7                Oman
143         Lithuania
67           Malaysia
27            Belarus
18     Macedonia, FYR
124              Iran
149           Vietnam
98              Sudan
156        Tajikistan
Name: country, dtype: object


In [39]:
# Elimina los valores duplicados basados en una columna Pais
pib_df = pib_df.drop_duplicates(subset=['country'])

In [40]:
pib_df.shape #Tabla sin valores duplicados

(166, 16)

[20%] Entendimiento de los datos: Genere un breve reporte de entendimiento de
datos en donde destaque las dimensiones del dataset, los tipos de datos que
contiene y el top 5 de los indicadores que considera más importantes para el
ejercicio de regresión propuesto. Por cada indicador, incluya algunos elementos
básicos de su comportamiento o distribución (análisis univariado). También incluya
algunos elementos complementarios que permitan entender la relación entre
pares de indicadores, el país o la región a la que pertenecen (análisis bivariado).

In [41]:
# Dimensiones del Dataset
pib_df.shape

(166, 16)

In [42]:
# Tipo de Datos que contiene
pib_df.dtypes

country                  object
incomeperperson         float64
alcconsumption          float64
armedforcesrate         float64
breastcancerper100th    float64
co2emissions            float64
femaleemployrate        float64
hivrate                 float64
internetuserate         float64
lifeexpectancy          float64
oilperperson            float64
polityscore             float64
relectricperperson      float64
suicideper100th         float64
employrate              float64
urbanrate               float64
dtype: object

Top 5 de los indicadores que considera más importantes para el ejercicio de regresión propuesto:

- Tasa de empleo femenina = (empleadas mayores de 15 años (% de la población). Porcentaje de población femenina mayor de 15 años que ha estado empleada durante el año determinado.)

- Usuario de Internet
- Esperanza de vida
- Tasa de VIH
- Emisiones de CO2

In [ ]:
#Sacar la media, la mediana el pais con mayor PIB y el Menor PIB y mirar a Colombia

In [56]:
pib_df = pib_df.sort_values('incomeperperson', ascending=False)
pib_df.head()

,country,incomeperperson,alcconsumption,armedforcesrate,breastcancerper100th,co2emissions,femaleemployrate,hivrate,internetuserate,lifeexpectancy,oilperperson,polityscore,relectricperperson,suicideper100th,employrate,urbanrate
166,Luxembourg,52301.58718,12.84,0.731336,82.5,5.920127e+08,45.900002,0.30,90.079527,79.963,NaN,NaN,1566.106139,12.405918,53.500000,82.44
44,Norway,39972.35277,8.35,0.720528,74.8,1.962704e+09,60.900002,0.10,93.277508,81.097,2.191226,10.0,7432.130852,10.823000,65.000000,77.48
42,Japan,39309.47886,7.79,0.361858,32.7,4.609221e+10,46.200001,0.06,77.638535,83.394,1.593223,10.0,2257.572233,18.946930,57.299999,66.48
32,Switzerland,37662.75125,11.41,0.524220,81.7,2.406741e+09,57.000000,0.40,82.166660,82.338,1.487412,10.0,2361.033336,13.239810,64.300003,73.48
177,United States,37491.17952,9.70,0.972378,101.1,3.340000e+11,56.000000,0.60,74.247572,78.531,2.738544,10.0,4542.848695,9.927033,62.299999,81.70


In [46]:
pib_df.isnull().sum()

country                   0
incomeperperson           3
alcconsumption            0
armedforcesrate           4
breastcancerper100th      1
co2emissions              2
femaleemployrate          1
hivrate                  22
internetuserate           3
lifeexpectancy            0
oilperperson            105
polityscore              10
relectricperperson       36
suicideper100th           0
employrate                1
urbanrate                 0
dtype: int64

In [ ]:
print(df['Numbers_One'].mean(skipna = True))

In [60]:
df = pd.DataFrame(pib_df, columns=['incomeperperson'])
# Identificar registros que son de tipo texto
registros_texto = df[df['incomeperperson'].apply(lambda x: isinstance(x, str))]
# Contar cuántos registros son de tipo texto
cantidad_registros_texto = len(registros_texto)
print(f'Cantidad de registros de tipo texto: {cantidad_registros_texto}')

Cantidad de registros de tipo texto: 0
